# Le fasi del progetto di analisi dei dati {#sec-freq-distribution}

**Prerequisiti**

- Leggere [Data Science: How is it Different To Statistics?](https://imstat.org/2014/09/04/data-science-how-is-it-different-to-statistics%E2%80%89/) di Hadley Wickham.
- Leggi [Veridical Data Science](https://vdsbook.com) [@yu2024veridical] focalizzandoti sul primo capitolo, che introduce le problematiche della data science, e sul quarto capitolo, che fornisce le linee guida dettagliate sull'organizzazione di un progetto di analisi dei dati.

**Concetti e competenze chiave**

**Preparazione del Notebook**

In [2]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import arviz as az
from pathlib import Path

In [3]:
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)
sns.set_theme(palette="colorblind")
az.style.use("arviz-darkgrid")
%config InlineBackend.figure_format = 'retina'

## Introduzione

Secondo @yu2024veridical, ogni progetto di analisi dei dati segue una combinazione delle seguenti fasi:

1. Formulazione del problema e raccolta dei dati.
2. Pulizia dei dati, preprocessing e analisi esplorativa.
3. Analisi predittiva e/o inferenziale.
4. Valutazione dei risultati.
5. Comunicazione dei risultati.

Mentre quasi tutti i progetti di data science attraversano le fasi 1-2 e 4-5, non tutti includono la fase 3.

## Fase 1: Formulazione del Problema e Raccolta dei Dati

La prima fase del ciclo di vita di un progetto di data science (DSLC) implica la formulazione di una domanda di ricerca che possa essere risolta utilizzando i dati disponibili. Questo potrebbe sembrare semplice, ma spesso la domanda iniziale è troppo vaga o non risolvibile. L'obiettivo è riformulare la domanda in modo tale che possa trovare una risposta utilizzando i dati a disposizione.

### Raccolta dei Dati

Alcuni progetti utilizzano dati esistenti (da repository pubblici, database interni o esperimenti passati), mentre altri richiedono la raccolta di nuovi dati. Ogni volta che è possibile, è necessario avere ben chiaro quali analisi statistiche verranno svolte *prima* di aver raccolto i dati. Se questo non viene fatto, può succedere che i dati raccolti non siano adeguati per rispondere alle domande di interesse, in quanto mancano informazioni cruciali, o vengono violate assunzioni richieste dai modelli statistici che si vogliono impiegare.

È importante sviluppare una comprensione dettagliata di come i dati sono stati raccolti e cosa significano i valori al loro interno. È altrettanto importante essere consapevoli degli strumenti e delle procedure utilizzate per la raccolta dei dati.

### Terminologia dei Dati

Ogni colonna di una tabella di dati (spesso chiamata semplicemente "dati" o "dataset") corrisponde a un diverso tipo di misurazione e viene denominata variabile, caratteristica, attributo o covariata dei dati.

Ogni variabile in un dataset ha tipicamente uno dei seguenti tipi:

- **Numerica**: Un valore continuo (ad es. l'importo di spesa), una durata (ad es. il numero di secondi che un paziente può stare in equilibrio su un piede, o il tempo che un visitatore trascorre sul tuo sito web), un conteggio (ad es. il numero di visitatori del tuo sito web in un periodo specificato, il numero di animali osservati in una determinata località), ecc.
- **Categorica**: Un insieme di gruppi o categorie finite/fisse con un insieme di opzioni predeterminate, come partito politico, dipartimento ospedaliero, paese, genere, ecc.
- **Date e orari**: Date e orari possono avere vari formati, come "01/01/2020 23:00:05" o "1 gen 2020".
- **Testo strutturato (breve)**: Testo con una struttura o lunghezza prestabilita, come il nome di una persona, un indirizzo postale, un indirizzo email, ecc.
- **Testo non strutturato (lungo)**: Un corpo di testo più ampio che non ha una struttura predefinita, come voci nei rapporti di patologia o note del medico, recensioni di film, tweet, post su Reddit, ecc.

La dimensione dei dati si riferisce al numero di variabili (colonne) che contiene (e talvolta anche al numero di righe che contiene). Pertanto, i "dati ad alta dimensione" si riferiscono tipicamente a dati con molte variabili (generalmente più di 100, anche se non esiste una soglia fissa oltre la quale i dati diventano "ad alta dimensione").

Ogni riga corrisponde a una particolare osservazione, unità osservazionale, unità di dati o punto dati (usiamo questi termini in modo intercambiabile). Queste sono le entità per cui vengono raccolte le misurazioni.

Questo formato, in cui i dati sono disposti in colonne (caratteristiche/variabili) e righe (unità osservazionali), è chiamato dati rettangolari o tabellari.

## Fase 2: Pulizia dei Dati e Analisi Esplorativa

### Pulizia dei Dati

Dopo aver definito una domanda e raccolto alcuni dati rilevanti, è il momento di pulire i dati. Un dataset pulito è ordinato, formattato in modo appropriato e ha voci non ambigue. La fase iniziale di pulizia dei dati consiste nell'identificare problemi con i dati (come formattazioni strane e valori non validi) e modificarli in modo che i valori siano validi e formattati in modo comprensibile sia per il computer che per noi. La pulizia dei dati è una fase incredibilmente importante di un progetto di data science perché non solo aiuta a garantire che i dati siano interpretati correttamente dal computer, ma aiuta anche a sviluppare una comprensione dettagliata delle informazioni contenute nei dati e delle loro limitazioni.

L'obiettivo della pulizia dei dati è creare una versione dei dati che rifletta nella maniera più fedele possibile la realtà e che sia interpretata correttamente dal computer. Per garantire che il computer utilizzi fedelmente le informazioni contenute nei dati, è necessario modificare i dati (scrivendo codice, non modificando il file dati grezzo stesso) in modo che siano in linea con ciò che il computer "si aspetta". Tuttavia, il processo di pulizia dei dati è necessariamente soggettivo e comporta fare assunzioni sulle quantità reali sottostanti misurate e decisioni su quali modifiche siano le più sensate.

### Preprocessing
Il preprocessing si riferisce al processo di modifica dei dati puliti per soddisfare i requisiti di un algoritmo specifico che si desidera applicare. Ad esempio, se si utilizza un algoritmo che richiede che le variabili siano sulla stessa scala, potrebbe essere necessario trasformarle, oppure, se si utilizza un algoritmo che non consente valori mancanti, potrebbe essere necessario imputarli o rimuoverli. Durante il preprocessing, potrebbe essere utile anche definire nuove caratteristiche/variabili utilizzando le informazioni esistenti nei dati, se si ritiene che queste possano essere utili per l'analisi.

Come per la pulizia dei dati, non esiste un unico modo corretto per pre-elaborare un dataset, e la procedura finale comporta tipicamente una serie di decisioni che dovrebbero essere documentate nel codice e nei file di documentazione.

### Analisi Esplorativa dei Dati
La fase successiva prevede un esame più approfondito dei dati mediante la creazione di tabelle informative, il calcolo di statistiche riassuntive come medie e mediane, e la produzione di visualizzazioni informative. Questa fase ha tipicamente due sottofasi. La prima sottofase, l'analisi esplorativa dei dati (EDA), implica lo sviluppo di riassunti numerici e visivi dei dati per comprendere meglio i dati e i modelli che contengono. La seconda sottofase, l'analisi esplicativa dei dati, consiste nel perfezionare le tabelle e i grafici esplorativi più informativi per comunicarli a un pubblico esterno.

## Fase 3: Analisi Predittiva e/o Inferenziale
Molte domande di data science sono formulate come problemi di previsione, dove l'obiettivo è utilizzare dati osservabili passati o presenti per prevedere qualcosa su dati futuri non visti, solitamente per aiutare a prendere decisioni nel mondo reale.

### Inferenza Basata sui Dati
Un altro tipo di problema basato sui dati che si può incontrare è quello dell'inferenza, che comporta l'apprendimento su una popolazione più ampia quantificando l'incertezza associata a una stima del parametro (come la "media del campione", che è una stima della "media della popolazione"). Le tecniche tradizionali di inferenza statistica includono il test delle ipotesi e gli intervalli di confidenza.

La maggior parte di questo corso è dedicata a fornire un'introduzione a come il problema dell'inferenza possa essere affrontato usando una prospettiva bayesiana.

## Fase 4: Valutazione dei Risultati
L'interpretazione dei risultati alla luce della domanda che ha motivato l'analisi è un passaggio cruciale. È necessario valutare qualitativamente i risultati utilizzando il pensiero critico e quantitativamente utilizzando gli standard correnti.

## Fase 5: Comunicazione dei Risultati 
L'ultima fase del ciclo di un progetto di data science implica la comunicazione dei risultati affinché possano essere utilizzati per prendere decisioni nel mondo reale. Questo potrebbe implicare la scrittura di un articolo di ricerca, la creazione di un report per un gruppo di lavoro, o la preparazione di alcune diapositive. La capacità di comunicare efficacemente i risultati dell'analisi alle persone che potrebbero utilizzarli è cruciale. Dopotutto, se hai condotto un'analisi approfondita ma non riesci a spiegare i risultati a nessuno, qual è stato il senso di condurre l'analisi in primo luogo?

La comunicazione deve essere personalizzata per il pubblico di riferimento. Piuttosto che presumere che il pubblico sia già familiare con il progetto, è necessario spiegare l'analisi e le figure in modo molto accurato e chiaro. Anche se il messaggio principale di una figura o diapositiva può essere ovvio per te, è buona pratica spiegare esplicitamente al pubblico come interpretarlo (senza usare gergo complesso).

## L'organizzazione del Progetto

Il primo requisito di un progetto di analisi dei dati è organizzare in maniera efficiente i vari file che verranno utilizzati: i file dei dati, i file di codice e la documentazione del progetto. Tutti i file relativi a un progetto di analisi dei dati devono essere contenuti in una singola cartella. @yu2024veridical propongono il seguente template per la struttura di un progetto:

![](../../figures/project_structure.png){width="27.5%"}

Le due cartelle principali sono:

- `data/`: contiene il dataset grezzo (ad esempio, `data.csv`) e una sottocartella con informazioni sulla documentazione dei dati (ad esempio, metainformazioni e definizioni dei dati sotto forma di codebook).
- `dslc_documentation/`: contiene file .qmd di Quarto (per R) o .ipynb di Jupyter Notebook (per Python) per condurre e documentare le esplorazioni e analisi basate su codice in ogni fase del progetto DSLC. Ogni nome di file ha un prefisso numerico per garantire che i file appaiano nell'ordine corretto. C'è anche una sottocartella `functions/` con script .R (R) o .py (Python) che contengono funzioni utilizzate in vari file di analisi.

Il file `README.md` riassume la struttura del progetto e descrive il contenuto di ogni file.

Una struttura di progetto come quella proposta da @yu2024veridical, in cui tutti i file sono contenuti in una singola cartella, offre un vantaggio significativo: la specificazione di tutti i percorsi dei file, ad esempio quelli necessari per la lettura dei dati, può essere fatta in maniera relativa, utilizzando come root la cartella del progetto. Questo assicura la portabilità del progetto tra diversi computer o utenti.

### I dati sulle aspettative negative nella depressione

Per illustrare gli aspetti dell'archiviazione dei dati sul computer e dell'importazione dei dati in Python, consideriamo i dati raccolti da @zetsche_2019future in uno studio che indaga le aspettative negative come meccanismo chiave nel mantenimento della depressione. I ricercatori hanno confrontato 30 soggetti con episodi depressivi con un gruppo di controllo di 37 individui sani, utilizzando il Beck Depression Inventory (BDI-II) per misurare la depressione.

Questo file CSV, così come tutti gli altri file di dati utilizzati in questa dispensa, è contenuto nella cartella `data` all'interno della cartella `psicometria`, che è la directory principale dell'intero progetto.

Con le seguenti istruzioni, specifico il percorso della directory principale del progetto in relazione alla mia directory personale:

In [4]:
# Get the home directory
home_directory = os.path.expanduser("~")
# Construct the path to the Quarto project directory
project_directory = os.path.join(home_directory, "_repositories", "psicometria")
print(project_directory)

/Users/corradocaudek/_repositories/psicometria


Avendo definito `project_directory` come root, diventa possibile specificare il percorso del file CSV che contiene i dati in relazione a `project_directory`.

In [5]:
# Definire il percorso del file CSV
file_path = os.path.join(project_directory, "data", "data.mood.csv")
print(file_path)

/Users/corradocaudek/_repositories/psicometria/data/data.mood.csv


Con la seguente istruzione posso dunque leggere i dati del file `data.mood.csv` in un DataFrame pandas.

In [6]:
df = pd.read_csv(file_path)

### Esaminare i dati

Per conoscere le dimensioni del DataFrame utilizziamo il metodo `.shape`.

In [7]:
df.shape

(1188, 44)

Il DataFrame ha 1188 righe e 44 colonne. Visualizziamo il nome delle colonne con il metodo `.columns`.

In [8]:
df.columns

Index(['Unnamed: 0', 'vpn_nr', 'esm_id', 'group', 'bildung', 'bdi',
       'nr_of_episodes', 'nobs_mood', 'trigger_counter', 'form', 'traurig_re',
       'niedergeschlagen_re', 'unsicher_re', 'nervos_re', 'glucklich_re',
       'frohlich_re', 'mood_sad.5', 'mood_fearful.5', 'mood_neg.5',
       'mood_happy.5', 'cesd_sum', 'rrs_sum', 'rrs_brood', 'rrs_reflect',
       'forecast_sad', 'forecast_fear', 'forecast_neg', 'forecast_happy',
       'recall_sad', 'recall_fear', 'recall_neg', 'recall_happy',
       'diff_neg.fore.5', 'diff_sad.fore.5', 'diff_fear.fore.5',
       'diff_happy.fore.5', 'diff_neg.retro.5', 'diff_sad.retro.5',
       'diff_fear.retro.5', 'diff_happy.retro.5', 'mood_sad5_tm1',
       'mood_neg5_tm1', 'mood_fearful5_tm1', 'mood_happy5_tm1'],
      dtype='object')

Dato che il DataFrame è troppo grande (1188 righe e 44 colonne), stampiamo sullo schermo le prime 5 righe.

In [9]:
df.head()

,Unnamed: 0,vpn_nr,esm_id,group,bildung,bdi,nr_of_episodes,nobs_mood,trigger_counter,form,...,diff_fear.fore.5,diff_happy.fore.5,diff_neg.retro.5,diff_sad.retro.5,diff_fear.retro.5,diff_happy.retro.5,mood_sad5_tm1,mood_neg5_tm1,mood_fearful5_tm1,mood_happy5_tm1
0,1,101,10,mdd,abitur,25.0,2.0,14,5,Forecasting,...,0.333333,-1.000000,0.250000,0.166667,0.333333,-1.000000,NaN,NaN,NaN,NaN
1,2,101,10,mdd,abitur,25.0,2.0,14,6,Forecasting,...,-0.666667,-0.333333,-0.416667,-0.166667,-0.666667,-0.333333,3.333333,3.000000,2.666667,3.000000
2,3,101,10,mdd,abitur,25.0,2.0,14,7,Forecasting,...,0.666667,-0.666667,1.250000,1.833333,0.666667,-0.666667,3.666667,3.666667,3.666667,2.333333
3,4,101,10,mdd,abitur,25.0,2.0,14,8,Forecasting,...,-0.333333,-0.666667,0.083333,0.500000,-0.333333,-0.666667,1.666667,2.000000,2.333333,2.666667
4,5,101,10,mdd,abitur,25.0,2.0,14,10,Forecasting,...,0.333333,-1.000000,0.416667,0.500000,0.333333,-1.000000,3.000000,3.166667,3.333333,2.666667


## Informazioni sull'Ambiente di Sviluppo

In [22]:
%load_ext watermark
%watermark -n -u -v -iv -w -m

Last updated: Thu Aug 01 2024

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit

arviz     : 0.18.0
matplotlib: 3.9.1
pandas    : 2.2.2
seaborn   : 0.13.2
numpy     : 1.26.4

Watermark: 2.4.3

